In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

import requests
from bs4 import BeautifulSoup
import re
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time

from selenium.common.exceptions import NoSuchElementException

In [2]:
chromedriver = '/Users/alex/Downloads/chromedriver 2'
browser = webdriver.Chrome(chromedriver)
#url = 'https://www.uncommongoods.com/gifts'
#browser.get(url)
#try:
    #myElem = WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.NAME, 'grid-card__inner')))
    #print("Page is ready!")
#except TimeoutException:
    #print("Loading took too much time!")

#respData = browser.page_source
#soup = BeautifulSoup(respData, 'html.parser')

In [3]:
def scroll_down():
    body = browser.find_element_by_css_selector('body')
    body.send_keys(Keys.PAGE_DOWN)
    
def get_data(url, category):
    browser.get(url + '/' + category)
    
    try:
        myElem = WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.NAME, 'grid-card__inner')))
        print("Page is ready!")
    except TimeoutException:
        print('Loading Done')    
    
    all_reviews = []
    ratings = []
    products = []
    reviewer_names = []
    reviewer_countries = []
    review_date = []
    review_time = []
    
    for i in range(10):
            scroll_down()
            
    respData = browser.page_source
    soup = BeautifulSoup(respData, 'html.parser')
    
    product_links = soup.find_all('a', class_='a-secondary')[:-13]
    time.sleep(2)
    
    product_count = 0

    for i in range(len(product_links)):
        link = url + product_links[i]['href']
        browser.get(link)
        for i in range(10):
            scroll_down()
        time.sleep(2)
        
        respData = browser.page_source
        soup = BeautifulSoup(respData, 'html.parser')
        time.sleep(2)

        try:
            product = soup.find('h1', itemprop='name').text
        except AttributeError:
            pass
        
        reviews = soup.find_all('p', class_='body-small', itemprop='reviewBody')
        for j in range(len(reviews)):
            all_reviews.append(reviews[j].text)
            ratings.append(soup.find_all('meta', itemprop='ratingValue')[j]['content'])
            products.append(product)
            reviewer_names.append(soup.find_all('span', style='white-space:nowrap;', itemprop='author')[j].text)
            reviewer_countries.append(
                soup.find_all('span', style='white-space:nowrap;', itemprop = lambda x: x != 'author')[j].text)
            review_date.append(soup.find_all('meta', itemprop='datePublished')[j]['content'][:10])
            review_time.append(soup.find_all('meta', itemprop='datePublished')[j]['content'][11:-5])
        
        product_count += 1
        browser.back()

        if product_count % 5 == 0:
            print('Gathered Reviews for 5 Products')
        if product_count > 50:
            break
            
    df = pd.DataFrame()
    df['Product'] = products
    df['Reviewer_Name'] = reviewer_names
    df['Reviewer_Country'] = reviewer_countries
    df['Review'] = all_reviews
    df['Review_Date'] = review_date
    df['Review_Time'] = review_time
    df['Rating'] = ratings
    

    return df

In [4]:
url = 'https://www.uncommongoods.com'
category = 'gifts'

data = get_data(url, category)

Loading Done
Gathered Reviews for 5 Products
Gathered Reviews for 5 Products
Gathered Reviews for 5 Products
Gathered Reviews for 5 Products
Gathered Reviews for 5 Products
Gathered Reviews for 5 Products
Gathered Reviews for 5 Products
Gathered Reviews for 5 Products
Gathered Reviews for 5 Products
Gathered Reviews for 5 Products


In [5]:
data.to_csv('UG_Review_Data.csv')